In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [3]:
api_wrapper=WikipediaAPIWrapper(top_k_pages=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name


'wikipedia'

In [4]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
import os
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
loader=WebBaseLoader("https://developers.google.com/machine-learning/resources/intro-llms/")
docs=loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)              
embedding = OllamaEmbeddings(model="tinyllama")
db = FAISS.from_documents(documents, embedding)
retriever = db.as_retriever()


USER_AGENT environment variable not set, consider setting it to identify your requests.
/var/folders/qq/bq9ljw7x0f1gz4c200b9lmx80000gn/T/ipykernel_51926/4193274693.py:12: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model="tinyllama")


In [5]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search","search for langsmith")
retriever_tool.name

'langsmith_search'

In [6]:
# Arxiv
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun 
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name


'arxiv'

In [7]:
tools=[wiki,retriever_tool,arxiv]
tools       

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/shivagupta/LLMs/myenv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='search for langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10c037880>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10be4a170>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x10bc369e0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_commun

In [10]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment")

os.environ["OPENAI_API_KEY"] = api_key
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0)

FileNotFoundError: [Errno 2] No such file or directory

In [ ]:
from langchain import hub   
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages



/Users/shivagupta/LLMs/myenv/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [11]:

### Agents

from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)


NameError: name 'llm' is not defined

In [12]:
# Agentic Executer
from langchain.agents import AgentExecutor
agent_executor= AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor



NameError: name 'agent' is not defined

In [13]:
agent_executor.invoke({"input":"tell me about langsmith"})

NameError: name 'agent_executor' is not defined